# import packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.ticker as plticker
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


# Importing libraries

In [2]:
world_cup = pd.read_csv("../datasets/World Cup 2018 Dataset.csv")
results = pd.read_csv("../datasets/results.csv")

In [3]:
winner = []
for i in range (len(results['home_team'])):
    if results['home_score'][i] > results['away_score'][i]:
        winner.append(results['home_team'][i])
    elif results['home_score'][i] < results['away_score'][i]:
        winner.append(results['away_team'][i])
    else:
        winner.append("Draw")
results['winning_team'] = winner

#adding goal differe column 

results['goal_difference']= np.absolute(results['home_score']-results['away_score'])
results.head()

date home_team away_team  home_score  away_score tournament     city  \
0  1872-11-30  Scotland   England           0           0   Friendly  Glasgow   
1  1873-03-08   England  Scotland           4           2   Friendly   London   
2  1874-03-07  Scotland   England           2           1   Friendly  Glasgow   
3  1875-03-06   England  Scotland           2           2   Friendly   London   
4  1876-03-04  Scotland   England           3           0   Friendly  Glasgow   

    country  neutral winning_team  goal_difference  
0  Scotland    False         Draw                0  
1   England    False      England                2  
2  Scotland    False     Scotland                1  
3   England    False         Draw                0  
4  Scotland    False     Scotland                3

In [4]:
# work with subset of data

df=results[(results['home_team']=='Nigeria') | (results['away_team'] =='Nigeria')]
nigeria = df.iloc[:]
nigeria.head()


date     home_team away_team  home_score  away_score tournament  \
2977  1949-10-08  Sierra Leone   Nigeria           0           2   Friendly   
3050  1950-05-28         Ghana   Nigeria           1           0   Friendly   
3219  1951-10-20       Nigeria     Ghana           5           0   Friendly   
3492  1953-10-11         Ghana   Nigeria           1           0   Friendly   
3654  1954-10-30       Nigeria     Ghana           3           0   Friendly   

          city       country  neutral winning_team  goal_difference  
2977  Freetown  Sierra Leone    False      Nigeria                2  
3050     Accra    Gold Coast    False        Ghana                1  
3219     Lagos       Nigeria    False      Nigeria                5  
3492     Accra    Gold Coast    False        Ghana                1  
3654     Lagos       Nigeria    False      Nigeria                3

In [5]:
year =[]
for row in nigeria['date']:
    year.append(int(row[:4]))
    
nigeria['match_year'] = year
nigeria_1930 = nigeria[nigeria.match_year >= 1930]
nigeria_1930.count()


date               548
home_team          548
away_team          548
home_score         548
away_score         548
tournament         548
city               548
country            548
neutral            548
winning_team       548
goal_difference    548
match_year         548
dtype: int64

In [6]:
worldcup_teams =['Australia', 'Iran','Japan','Korea Republic',
                 'Saudi Arabia', 'Egypt','Morocco',
                 'Nigeria', 'Senegal','Tunisia',
                 'Costa Rica','Mexico', 'Panama','Argentina'
                 ,'Brazil', 'Colombia' 'Peru',
                 'Uruguay', 'Belgium', 'croatia', 'Denmark',
                 'England','France', 'Iceland',
                 'Poland', 'Portugal','Russia',
                 'Serbia','Spain', 'Germany', 'sweden',
                 'switzerland']

In [7]:
len(worldcup_teams)

31

In [8]:
df_teams_home = results[results['home_team'].isin(worldcup_teams)]
df_teams_away = results[results['away_team'].isin(worldcup_teams)]
df_teams =pd.concat ((df_teams_home, df_teams_away)) 
df_teams.drop_duplicates() 
df_teams.count()

date               18058
home_team          18058
away_team          18058
home_score         18058
away_score         18058
tournament         18058
city               18058
country            18058
neutral            18058
winning_team       18058
goal_difference    18058
dtype: int64

In [9]:
#create an year column to drop games before 1930 
year=[]
for row in df_teams ['date']: 
    year.append (int (row[:4]))
df_teams['match_year']=year
df_teams_1930 = df_teams[df_teams.match_year>=1930]
df_teams_1930.head()

date home_team       away_team  home_score  away_score tournament  \
1230  1930-01-01     Spain  Czechoslovakia           1           0   Friendly   
1231  1930-01-12  Portugal  Czechoslovakia           1           0   Friendly   
1237  1930-02-23  Portugal          France           2           0   Friendly   
1238  1930-03-02   Germany           Italy           0           2   Friendly   
1240  1930-03-23    France     Switzerland           3           3   Friendly   

                   city   country  neutral winning_team  goal_difference  \
1230          Barcelona     Spain    False        Spain                1   
1231             Lisbon  Portugal    False     Portugal                1   
1237              Porto  Portugal    False     Portugal                2   
1238  Frankfurt am Main   Germany    False        Italy                2   
1240           Colombes    France    False         Draw                0   

      match_year  
1230        1930  
1231        1930  
1237        1930  
1238        1930  
1240        1930

In [10]:
#drop col that they dont influence matches
df_teams_1930 = df_teams.drop(['date','home_score','away_score',
                              'tournament','city','country'])
df_teams_1930.head()

date home_team away_team  home_score  away_score tournament    city  \
1   1873-03-08   England  Scotland           4           2   Friendly  London   
3   1875-03-06   England  Scotland           2           2   Friendly  London   
6   1877-03-03   England  Scotland           1           3   Friendly  London   
10  1879-01-18   England     Wales           2           1   Friendly  London   
11  1879-04-05   England  Scotland           5           4   Friendly  London   

    country  neutral winning_team  goal_difference  match_year  
1   England    False      England                2        1873  
3   England    False         Draw                0        1875  
6   England    False     Scotland                2        1877  
10  England    False      England                1        1879  
11  England    False      England                1        1879

In [11]:
#build a model

df_teams_1930 = df_teams_1930.reset_index(drop=True)
df_teams_1930.loc[df_teams_1930.winning_team == df_teams_1930.home_team,'winning_team']=2
df_teams_1930.loc[df_teams_1930.winning_team == 'Draw','winning_team']=1
df_teams_1930.loc[df_teams_1930.winning_team ==df_teams_1930.away_team,'winning_team']=0
df_teams_1930.head()

date home_team away_team  home_score  away_score tournament    city  \
0  1873-03-08   England  Scotland           4           2   Friendly  London   
1  1875-03-06   England  Scotland           2           2   Friendly  London   
2  1877-03-03   England  Scotland           1           3   Friendly  London   
3  1879-01-18   England     Wales           2           1   Friendly  London   
4  1879-04-05   England  Scotland           5           4   Friendly  London   

   country  neutral  winning_team  goal_difference  match_year  
0  England    False             2                2        1873  
1  England    False             1                0        1875  
2  England    False             0                2        1877  
3  England    False             2                1        1879  
4  England    False             2                1        1879

In [65]:
df_teams_1930[(df_teams_1930['match_year']==1989) & (df_teams_1930['home_team'] == "Argentina")]



date  home_team away_team  home_score  away_score    tournament  \
4287   1989-07-02  Argentina     Chile           1           0  Copa América   
4290   1989-07-04  Argentina   Ecuador           0           0  Copa América   
4295   1989-07-10  Argentina   Bolivia           0           0  Copa América   
4297   1989-07-14  Argentina   Uruguay           0           2  Copa América   
13529  1989-07-14  Argentina   Uruguay           0           2  Copa América   

                 city country  neutral  winning_team  goal_difference  \
4287          Goiânia  Brazil     True             2                1   
4290          Goiânia  Brazil     True             1                0   
4295          Goiânia  Brazil     True             1                0   
4297   Rio de Janeiro  Brazil     True             0                2   
13529  Rio de Janeiro  Brazil     True             0                2   

       match_year  
4287         1989  
4290         1989  
4295         1989  
4297         1989  
13529        1989

In [59]:
final =pd.get_dummies(df_teams_1930,prefix=['home_team','away_team'],columns=['home_team','away_team'])
final= final.drop(['date','tournament','city','country','neutral'],axis=1)
#seprate the X and y

X=final.drop(['winning_team'],axis=1)
y=final['winning_team']
y=y.astype('int')

x_train,x_test,y_train,y_test =train_test_split(X,y,test_size=0.30,random_state=42)

In [73]:
final.head()


home_score  away_score  winning_team  goal_difference  match_year  \
0           4           2             2                2        1873   
1           2           2             1                0        1875   
2           1           3             0                2        1877   
3           2           1             2                1        1879   
4           5           4             2                1        1879   

   home_team_Afghanistan  home_team_Albania  home_team_Algeria  \
0                      0                  0                  0   
1                      0                  0                  0   
2                      0                  0                  0   
3                      0                  0                  0   
4                      0                  0                  0   

   home_team_Andorra  home_team_Angola         ...          \
0                  0                 0         ...           
1                  0                 0         ...           
2                  0                 0         ...           
3                  0                 0         ...           
4                  0                 0         ...           

   away_team_Venezuela  away_team_Vietnam  away_team_Vietnam Republic  \
0                    0                  0                           0   
1                    0                  0                           0   
2                    0                  0                           0   
3                    0                  0                           0   
4                    0                  0                           0   

   away_team_Wales  away_team_Western Australia  away_team_Yemen  \
0                0                            0                0   
1                0                            0                0   
2                0                            0                0   
3                1                            0                0   
4                0                            0                0   

   away_team_Yemen DPR  away_team_Yugoslavia  away_team_Zambia  \
0                    0                     0                 0   
1                    0                     0                 0   
2                    0                     0                 0   
3                    0                     0                 0   
4                    0                     0                 0   

   away_team_Zimbabwe  
0                   0  
1                   0  
2                   0  
3                   0  
4                   0  

[5 rows x 418 columns]

In [74]:
logreg= LogisticRegression()
logreg.fit(x_train,y_train)
print(logreg.score(x_test,y_test))

1.0


In [75]:
world_cup

Team Group  Previous \nappearances  Previous \ntitles  \
0         Russia     A                    10.0                0.0   
1   Saudi Arabia     A                     4.0                0.0   
2          Egypt     A                     2.0                0.0   
3        Uruguay     A                    12.0                2.0   
4        Porugal     B                     6.0                0.0   
5          Spain     B                    14.0                1.0   
6        Morocco     B                     4.0                0.0   
7           IRAN     B                     4.0                0.0   
8         France     C                    14.0                1.0   
9      Australia     C                     4.0                0.0   
10          Peru     C                     4.0                0.0   
11       Denmark     C                     4.0                0.0   
12     Argentina     D                    16.0                2.0   
13       Iceland     D                     0.0                0.0   
14       Croatia     D                     4.0                0.0   
15       Nigeria     D                     5.0                0.0   
16        Brazil     E                    20.0                5.0   
17   Switzerland     E                    10.0                0.0   
18     Costarica     E                     4.0                0.0   
19        Serbia     E                    11.0                0.0   
20       Germany     F                    18.0                4.0   
21        Mexico     F                    15.0                0.0   
22        Sweden     F                    11.0                0.0   
23         Korea     F                     9.0                0.0   
24       Belgium     G                    12.0                0.0   
25        Panama     G                     0.0                0.0   
26       Tunisia     G                     4.0                0.0   
27       England     G                    14.0                1.0   
28        Poland     H                     7.0                0.0   
29       Senegal     H                     1.0                0.0   
30      Columbia     H                     5.0                0.0   
31         Japan     H                     5.0                0.0   
32           NaN   NaN                     NaN                NaN   

    Previous\n finals  Previous\n semifinals  Current \nFIFA rank  \
0                 0.0                    1.0                 65.0   
1                 0.0                    0.0                 63.0   
2                 0.0                    0.0                 31.0   
3                 2.0                    5.0                 21.0   
4                 0.0                    2.0                  3.0   
5                 1.0                    2.0                  6.0   
6                 0.0                    0.0                 40.0   
7                 0.0                    0.0                 32.0   
8                 2.0                    5.0                  9.0   
9                 0.0                    0.0                 39.0   
10                0.0                    0.0                 11.0   
11                0.0                    0.0                 12.0   
12                5.0                    5.0                  4.0   
13                0.0                    0.0                 22.0   
14                0.0                    1.0                 17.0   
15                0.0                    0.0                 50.0   
16                7.0                   11.0                  2.0   
17                0.0                    0.0                  8.0   
18                0.0                    0.0                 26.0   
19                0.0                    2.0                 37.0   
20                8.0                   13.0                  1.0   
21                0.0                    0.0                 16.0   
22                1.0                    4.0                 18.0   
23       